In [231]:
import numpy as np
import tensorflow.compat.v1 as tf
from tensorflow.python.framework import ops
tf.disable_v2_behavior() 
import pandas as pd
from sklearn import preprocessing
import sklearn
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline


In [232]:
def create_placeholder(Xlen, Ylen):
    X = tf.placeholder(tf.float64, shape=[Xlen,None], name = "X")
    Y = tf.placeholder(tf.float64, shape = [Ylen,None],name = "Y")
    
    return X,Y

In [233]:
def intialize_parameters():
    W1 = tf.get_variable("W1", [25,30],initializer = tf.random_normal_initializer, dtype = "float64")
    B1 = tf.get_variable("B1", [25,1],initializer = tf.zeros_initializer(), dtype = "float64")
   
    W2 = tf.get_variable("W2", [2,25],initializer = tf.random_normal_initializer, dtype = "float64")
    B2 = tf.get_variable("B2", [2,1],initializer = tf.zeros_initializer(), dtype = "float64")
    
    parameters = {"W1": W1,
                  "W2": W2,
                  "B1": B1,
                  "B2" :B2}
    
    return parameters

In [234]:
def one_hot_matrix(Y, C):

    C = tf.constant(C, name='C')
    Y = tf.one_hot(indices=Y, depth=C, axis=0)
    
    
    sess = tf.Session()
    Y_encoded = sess.run(Y)
    sess.close()
    
    return Y_encoded
    

In [282]:
def Cost_fn(Y_h, Y, parameter):
    
    #cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=Z2, labels=Y))
    loss = (tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=tf.transpose(Y_h), labels= tf.transpose(Y))) +
    0.01*tf.nn.l2_loss(parameter["W1"]) +
    0.01*tf.nn.l2_loss(parameter["B1"]) +
    0.01*tf.nn.l2_loss(parameter["W2"]) +
    0.01*tf.nn.l2_loss(parameter["B2"]))
    
    return loss

In [283]:
def Forward_propogation(X, parameters):
    
    W1 = parameters["W1"]
    W2 = parameters["W2"]
    B1 = parameters["B1"]
    B2 = parameters["B2"]
    
    #print("w1 * X =", W1.shape, X.shape)
    mul1= tf.matmul(W1,X)
    #print("mul1 shape = ", mul1.shape)
    Z1 = tf.add(mul1 , B1)
    #print("z1 -", Z1.shape)
    A1 = tf.nn.relu(Z1)
    
    #print("W2 * A1 = ", W2.shape, A1.shape)
    mul2= tf.matmul(W2,A1)
    #print("mul2 shape = ", mul2.shape)
    
    Z2 = tf.add(mul2, B2)
    A2 = tf.nn.softmax(Z2)
    
    
    return A2
    

In [287]:
def model(X_train, X_test, Y_train, Y_test, learning_rate =0.1, print_cost =True, num_iters = 10000):
    
    
    ops.reset_default_graph() 
    costList = []
    Rows, Cols = X_train.shape
    X, Y = create_placeholder(X_train.shape[0], Y_train.shape[0])
    #print("y shape = " Y.shape)
    
    parameters = intialize_parameters()
    
    Y_h = Forward_propogation(X, parameters)
    
    cost = Cost_fn(Y_h, Y, parameters)
    
    optimizer = tf.train.GradientDescentOptimizer(learning_rate= learning_rate).minimize(cost)
    
    init = tf.initialize_all_variables()
    
    with tf.Session() as ss:
        
        ss.run(init)
        
        for i in range(num_iters):
            _, costV = ss.run([optimizer,cost], feed_dict={X:X_train,Y:Y_train})
            print ("Cost after", i," = ", costV )
            costList.append(costV)
        
    
                   
    #plt.plot(np.squeeze(costList))
    #plt.ylabel('cost')
    #plt.xlabel('iterations (per tens)')
    #plt.title("Learning rate =" + str(learning_rate))
    #plt.show()

    
    
    #parameters = ss.run(parameters)
        print("Parameters have been trained!")

        # Calculate the correct predictions
        correct_prediction = tf.equal(tf.argmax(Y_h), tf.argmax(Y))

        # Calculate accuracy on the test set
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
        print("Train Accuracy:", accuracy.eval({X: X_train, Y: Y_train}))
        print("Test Accuracy:", accuracy.eval({X: X_test, Y: Y_test}))
        
    return parameters
    
    

In [288]:
df = pd.read_csv("data.csv")
df.drop('id', inplace = True, axis =1)
df.drop('Unnamed: 32',axis=1,inplace=True)

df['diagnosis'] = df['diagnosis'].map({'M':1, 'B':0})
train, test = train_test_split(df, test_size=0.3)

X_train   = train.drop('diagnosis', axis = 1, inplace = False)
scaler    = preprocessing.StandardScaler()
scaled_df = scaler.fit_transform(X_train)
X_train   = pd.DataFrame(scaled_df)

X_test = test.drop('diagnosis', axis = 1, inplace = False)
scaler    = preprocessing.StandardScaler()
scaled_df = scaler.fit_transform(X_test)
X_test   = pd.DataFrame(scaled_df)


Y_train = train['diagnosis']
Y_test = test['diagnosis']

X_train = np.array(np.transpose(X_train))
X_test =  np.array(np.transpose(X_test))
Y_train = np.array(np.transpose(Y_train))
Y_test =  np.array(np.transpose(Y_test))



Y_train = one_hot_matrix(Y_train,2)
Y_test  = one_hot_matrix(Y_test,2)


print("X_train shape : ", X_train.shape)
print("Y_train shape : ", Y_train.shape)
print("X_test shape : ", X_test.shape)
print("Y_test shape : ", Y_test.shape)

X_train shape :  (30, 398)
Y_train shape :  (2, 398)
X_test shape :  (30, 171)
Y_test shape :  (2, 171)


In [289]:
tf.reset_default_graph()
parameters = model(X_train, X_test, Y_train, Y_test)


Cost after 0  =  4.572180134437807
Cost after 1  =  4.564398259517337
Cost after 2  =  4.55663203780097
Cost after 3  =  4.548881484819039
Cost after 4  =  4.541146615195576
Cost after 5  =  4.533427442181304
Cost after 6  =  4.525723977217804
Cost after 7  =  4.518036229533023
Cost after 8  =  4.510364205783489
Cost after 9  =  4.502707909749448
Cost after 10  =  4.495067342088419
Cost after 11  =  4.487442500150424
Cost after 12  =  4.479833377856067
Cost after 13  =  4.472239965636345
Cost after 14  =  4.464662250431351
Cost after 15  =  4.457100215743284
Cost after 16  =  4.4495538417379965
Cost after 17  =  4.4420231053883645
Cost after 18  =  4.434507980652267
Cost after 19  =  4.427008438677782
Cost after 20  =  4.419524448028348
Cost after 21  =  4.4120559749209685
Cost after 22  =  4.404602983471195
Cost after 23  =  4.397165435939281
Cost after 24  =  4.389743292972692
Cost after 25  =  4.382336513840995
Cost after 26  =  4.374945056660004
Cost after 27  =  4.367568878602702


Cost after 268  =  2.960857081816057
Cost after 269  =  2.95632034995799
Cost after 270  =  2.9517926841630295
Cost after 271  =  2.9472740662809738
Cost after 272  =  2.9427644781981237
Cost after 273  =  2.9382639018372765
Cost after 274  =  2.933772319157574
Cost after 275  =  2.9292897121544494
Cost after 276  =  2.9248160628595605
Cost after 277  =  2.9203513533407137
Cost after 278  =  2.9158955657017893
Cost after 279  =  2.9114486820826717
Cost after 280  =  2.907010684659174
Cost after 281  =  2.9025815556429646
Cost after 282  =  2.8981612772815004
Cost after 283  =  2.893749831857951
Cost after 284  =  2.889347201691126
Cost after 285  =  2.8849533691354052
Cost after 286  =  2.88056831658067
Cost after 287  =  2.8761920264522303
Cost after 288  =  2.8718244812107536
Cost after 289  =  2.867465663352192
Cost after 290  =  2.8631155554077137
Cost after 291  =  2.8587741399436397
Cost after 292  =  2.8544413995614657
Cost after 293  =  2.8501173168977796
Cost after 294  =  2.8

Cost after 520  =  2.061946591192012
Cost after 521  =  2.0592061665647226
Cost after 522  =  2.056471222962934
Cost after 523  =  2.053741749492269
Cost after 524  =  2.0510177352797245
Cost after 525  =  2.048299169473623
Cost after 526  =  2.045586041243559
Cost after 527  =  2.0428783397803496
Cost after 528  =  2.0401760542959804
Cost after 529  =  2.037479174023705
Cost after 530  =  2.034787688217914
Cost after 531  =  2.03210158615345
Cost after 532  =  2.0294208571264782
Cost after 533  =  2.0267454904540796
Cost after 534  =  2.024075475474207
Cost after 535  =  2.021410801545636
Cost after 536  =  2.0187514580479173
Cost after 537  =  2.0160974343813276
Cost after 538  =  2.0134487199668216
Cost after 539  =  2.010805304246024
Cost after 540  =  2.0081671766812637
Cost after 541  =  2.0055343267550736
Cost after 542  =  2.0029067439706725
Cost after 543  =  2.0002844178517023
Cost after 544  =  1.9976673379422212
Cost after 545  =  1.9950554938066438
Cost after 546  =  1.992

Cost after 781  =  1.5044005428279281
Cost after 782  =  1.5027762544495467
Cost after 783  =  1.501155219266226
Cost after 784  =  1.499537430762755
Cost after 785  =  1.4979228824368538
Cost after 786  =  1.4963115677991483
Cost after 787  =  1.4947034803731438
Cost after 788  =  1.4930986136952002
Cost after 789  =  1.4914969613145086
Cost after 790  =  1.4898985167930596
Cost after 791  =  1.4883032737056276
Cost after 792  =  1.4867112256397375
Cost after 793  =  1.4851223661957431
Cost after 794  =  1.4835366889891104
Cost after 795  =  1.4819541876429334
Cost after 796  =  1.4803748557955014
Cost after 797  =  1.4787986870973593
Cost after 798  =  1.4772256752101676
Cost after 799  =  1.4756558137751592
Cost after 800  =  1.47408909651728
Cost after 801  =  1.472525517137707
Cost after 802  =  1.470965069361245
Cost after 803  =  1.4694077469182396
Cost after 804  =  1.4678535435325415
Cost after 805  =  1.4663024529552025
Cost after 806  =  1.4647544689496816
Cost after 807  = 

Cost after 1032  =  1.1837159848630259
Cost after 1033  =  1.1827338649269115
Cost after 1034  =  1.181753710162629
Cost after 1035  =  1.1807755166326162
Cost after 1036  =  1.1797992804073316
Cost after 1037  =  1.1788249975652403
Cost after 1038  =  1.1778526641927962
Cost after 1039  =  1.1768822763844262
Cost after 1040  =  1.1759138302425158
Cost after 1041  =  1.1749473218773931
Cost after 1042  =  1.1739827474073101
Cost after 1043  =  1.1730201029584304
Cost after 1044  =  1.1720593846648115
Cost after 1045  =  1.171100588668389
Cost after 1046  =  1.1701437111189616
Cost after 1047  =  1.1691887481741734
Cost after 1048  =  1.1682356960331495
Cost after 1049  =  1.167284550929375
Cost after 1050  =  1.1663353090115243
Cost after 1051  =  1.1653879668580516
Cost after 1052  =  1.1644425202434423
Cost after 1053  =  1.1634989653462078
Cost after 1054  =  1.1625572983784493
Cost after 1055  =  1.161617515415451
Cost after 1056  =  1.1606796124328411
Cost after 1057  =  1.1597435

...

Cost after 8954  =  0.6931473712491435
Cost after 8955  =  0.6931473709451677
Cost after 8956  =  0.6931473706416925
Cost after 8957  =  0.693147370338717
Cost after 8958  =  0.6931473700362403
Cost after 8959  =  0.6931473697342618
Cost after 8960  =  0.6931473694327804
Cost after 8961  =  0.6931473691317955
Cost after 8962  =  0.6931473688313061
Cost after 8963  =  0.6931473685313114
Cost after 8964  =  0.6931473682318106
Cost after 8965  =  0.693147367932803
Cost after 8966  =  0.6931473676342873
Cost after 8967  =  0.6931473673362629
Cost after 8968  =  0.6931473670387293
Cost after 8969  =  0.6931473667416851
Cost after 8970  =  0.6931473664451299
Cost after 8971  =  0.6931473661490626
Cost after 8972  =  0.6931473658534827
Cost after 8973  =  0.6931473655583891
Cost after 8974  =  0.6931473652637806
Cost after 8975  =  0.6931473649696571
Cost after 8976  =  0.6931473646760176
Cost after 8977  =  0.693147364382861
Cost after 8978  =  0.6931473640901865
Cost after 8979  =  0.693147

Cost after 9221  =  0.6931473054757532
Cost after 9222  =  0.6931473052793788
Cost after 9223  =  0.6931473050833235
Cost after 9224  =  0.6931473048875857
Cost after 9225  =  0.6931473046921657
Cost after 9226  =  0.693147304497063
Cost after 9227  =  0.6931473043022768
Cost after 9228  =  0.6931473041078067
Cost after 9229  =  0.6931473039136521
Cost after 9230  =  0.6931473037198121
Cost after 9231  =  0.6931473035262868
Cost after 9232  =  0.6931473033330756
Cost after 9233  =  0.6931473031401777
Cost after 9234  =  0.6931473029475927
Cost after 9235  =  0.6931473027553202
Cost after 9236  =  0.6931473025633597
Cost after 9237  =  0.6931473023717106
Cost after 9238  =  0.693147302180372
Cost after 9239  =  0.6931473019893439
Cost after 9240  =  0.6931473017986258
Cost after 9241  =  0.6931473016082167
Cost after 9242  =  0.6931473014181163
Cost after 9243  =  0.6931473012283245
Cost after 9244  =  0.6931473010388401
Cost after 9245  =  0.6931473008496629
Cost after 9246  =  0.69314

Cost after 9485  =  0.6931472632445574
Cost after 9486  =  0.6931472631162816
Cost after 9487  =  0.6931472629882109
Cost after 9488  =  0.6931472628603454
Cost after 9489  =  0.6931472627326843
Cost after 9490  =  0.6931472626052277
Cost after 9491  =  0.6931472624779751
Cost after 9492  =  0.6931472623509247
Cost after 9493  =  0.6931472622240764
Cost after 9494  =  0.6931472620974312
Cost after 9495  =  0.6931472619709886
Cost after 9496  =  0.6931472618447485
Cost after 9497  =  0.69314726171871
Cost after 9498  =  0.6931472615928734
Cost after 9499  =  0.6931472614672378
Cost after 9500  =  0.6931472613418038
Cost after 9501  =  0.6931472612165698
Cost after 9502  =  0.6931472610915363
Cost after 9503  =  0.6931472609667029
Cost after 9504  =  0.6931472608420688
Cost after 9505  =  0.693147260717634
Cost after 9506  =  0.6931472605933984
Cost after 9507  =  0.6931472604693614
Cost after 9508  =  0.6931472603455225
Cost after 9509  =  0.6931472602218818
Cost after 9510  =  0.693147

Cost after 9749  =  0.693147235582138
Cost after 9750  =  0.693147235497849
Cost after 9751  =  0.693147235413693
Cost after 9752  =  0.6931472353296699
Cost after 9753  =  0.6931472352457794
Cost after 9754  =  0.6931472351620213
Cost after 9755  =  0.6931472350783954
Cost after 9756  =  0.6931472349949016
Cost after 9757  =  0.6931472349115393
Cost after 9758  =  0.693147234828309
Cost after 9759  =  0.6931472347452096
Cost after 9760  =  0.6931472346622416
Cost after 9761  =  0.6931472345794043
Cost after 9762  =  0.6931472344966979
Cost after 9763  =  0.6931472344141217
Cost after 9764  =  0.693147234331676
Cost after 9765  =  0.6931472342493603
Cost after 9766  =  0.6931472341671743
Cost after 9767  =  0.6931472340851181
Cost after 9768  =  0.6931472340031912
Cost after 9769  =  0.6931472339213937
Cost after 9770  =  0.6931472338397252
Cost after 9771  =  0.6931472337581851
Cost after 9772  =  0.693147233676774
Cost after 9773  =  0.6931472335954912
Cost after 9774  =  0.693147233

Train Accuracy: 0.92462313
Test Accuracy: 0.94152045


Instructions for updating:
Use `tf.global_variables_initializer` instead.


<tf.Operation 'init' type=NoOp>